In [1]:
from pyphare.simulator.simulator import Simulator
from test_density import config

In [15]:
sim = config()

validating dim=1


In [ ]:
Simulator(sim).run().reset()

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from pyphare.pharesee.run import Run

mpl.use("TkAgg")

In [4]:
run_path = os.path.join(os.curdir, "nCheck")
time = 0.0

In [5]:
r = Run(run_path)

In [6]:
Nm = r.GetMassDensity(time)

In [ ]:
Nm.level(0).patches[0].patch_datas

In [ ]:
from pyphare.pharesee.hierarchy import hierarchy_utils

In [ ]:
newN = hierarchy_utils.rename(Nm, ["rho"])

In [ ]:
newN.level(0).patches[0].patch_datas

In [ ]:
fig, ax = plt.subplots()

In [ ]:
newN.plot(ax=ax)

In [ ]:
plt.show()

In [7]:
hier_computed = r.GetMassDensity(time)

In [49]:
L = sim.simulation_domain()

In [50]:
def densityMain(x):
    return 1.

def densityBeam(x):
    u = x/L-0.5
    return np.exp(-u**2)

m_Main = 2
m_Beam = 3

In [51]:
def _compute_ions_mass_density_profile(patch_datas, **kwargs):
    ref_name = next(iter(patch_datas.keys()))

    x_ = patch_datas[ref_name].x
    print(type(x_), x_)
    dset = m_Main*densityMain(x_)+m_Beam*densityBeam(x_)
    
    # dset = patch_datas["value"][:]
    return ({"name": "value", "data": dset, "centering": patch_datas[ref_name].centerings},)

In [52]:
from pyphare.pharesee.hierarchy.hierarchy_utils import compute_hier_from
hier_expected = compute_hier_from(_compute_ions_mass_density_profile, hier_computed)

<class 'numpy.ndarray'> [-0.4 -0.2  0.   0.2  0.4  0.6  0.8  1.   1.2  1.4  1.6  1.8  2.   2.2
  2.4  2.6  2.8  3.   3.2  3.4  3.6  3.8  4.   4.2  4.4  4.6  4.8  5.
  5.2  5.4  5.6  5.8  6.   6.2  6.4  6.6  6.8  7.   7.2  7.4  7.6  7.8
  8.   8.2  8.4  8.6  8.8  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4]
<class 'numpy.ndarray'> [ 9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.  12.2
 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8 15.
 15.2 15.4 15.6 15.8 16.  16.2 16.4 16.6 16.8 17.  17.2 17.4 17.6 17.8
 18.  18.2 18.4 18.6 18.8 19.  19.2 19.4 19.6 19.8 20.  20.2 20.4]


In [53]:
from pyphare.pharein import global_vars

In [54]:
global_vars.sim